
Importar librerias<br>


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
matplotlib.use('TkAgg')
from sklearn import preprocessing  # Antes de importar hay que instalar sklearn


S5.T0. Importar la base de datos a PyCharm<br>
- Guarda la base de datos en un dataframe llamado "df"<br>
- El dtype de las columnas indicadas a continuaciÃ³n las convertimos de float64 a float32 para reducir memoria<br>
- En sesiones posteriores, si actualizamos la BBDD y eliminamos alguna de estas columnas, hay que actualizar el dtype de la columna en el cÃ³digo<br>


In [ ]:
df = pd.read_csv(r'E:\Laptop\BBDD_ST3.csv')  # Definir ruta de la BBDD
df["CODI_POSTAL"] = df["CODI_POSTAL"].astype('str')  # Convertir a string el cÃ³digo postal
df["VALOR AILLAMENTS CTE"] = df["VALOR AILLAMENTS CTE"].astype('str')  # Convertir a string el valor de aislamiento
df["VALOR FINESTRES CTE"] = df["VALOR FINESTRES CTE"].astype('str')  # Convertir a string el valor de las ventanas
float64_cols = df.select_dtypes(include='float64').columns.tolist()  # Seleccionar las columnas de tipo float64
df[float64_cols] = df[float64_cols].astype('float32')  # Convertir a float32 las columnas seleccionadas
int64_cols = df.select_dtypes(include='int64').columns.tolist()  # Seleccionar las columnas de tipo int64
df[int64_cols] = df[int64_cols].astype('int32')  # Convertir a int32 las columnas seleccionadas


S5.T1. SelecciÃ³n inicial de variables de entrada y salida del modelo<br>
- Verifica que tu dataframe no contiene NaNs<br>
- Selecciona la variable de salida (se recomienda elegir la misma que en la sesiÃ³n de trabajo 4: "QualificaciÃ³ de consum d'energia primaria no renovable" o "Qualificacio dâ€™emissions de CO2"). Elimina todos los demÃ¡s outputs.<br>
- Elimina aquellas variables de entrada que no presenten correlaciÃ³n con el output elegido. En caso de duda, mejor mantener la variable. <br>


TODO: VERIFICACIÃ“N PRESENCIA NaNs

In [ ]:
    # Comandos Ãºtiles para ver si hay NaNs en el dataframe

In [ ]:
df.info()
Num_Nans = np.sum(df[df.columns].isnull())

    # TAREA: Verifica que no nay NaNs.

TODO: SelecciÃ³n variable de salida. Eliminar las demÃ¡s variables de salida.

In [ ]:
    # EJEMPLO ELIMINACIÃ“N VARIABLE SALIDA 1: Energia primÃ ria no renovable

In [ ]:
df = df.drop("Energia primÃ ria no renovable", axis=1)

    # TAREA: Elimina aquellas variables de salida que no vas a evaluar (3 de 4). Escribe el cÃ³digo a continuaciÃ³n:

TODO: EliminaciÃ³n de variables de entrada categÃ³ricas con un nÃºmero muy elevado de campos (mÃ¡s de 1000)

In [ ]:
    # EJEMPLO ELIMINACIÃ“N: POBLACIO

In [ ]:
df = df.drop("POBLACIO", axis=1)

    # TAREA: Elimina aquellas variables de entrada categÃ³ricas con un nÃºmero muy elevado de campos (mÃ¡s de 1000).
    # Escribe el cÃ³digo a continuaciÃ³n:

In [ ]:
df = df.drop("CODI_POSTAL", axis=1)


S5.T2. Escalado/NormalizaciÃ³n de las variables continuas<br>


EJEMPLO ALTERNATIVA 1: MIXMAXSCALER EN METRES_CADASTRE

In [ ]:
sns.kdeplot(data=[df['METRES_CADASTRE']])  # Verificar distribuciÃ³n de la variable "METRES_CADASTRE"
scaler = preprocessing.MinMaxScaler()  # Definir el escalador MinMaxScaler
df[['METRES_CADASTRE']] = scaler.fit_transform(df[['METRES_CADASTRE']])  # Escalar la variable "METRES_CADASTRE"

EJEMPLO ALTERNATIVA 2: ROBUSTSCALER EN METRES_CADASTRE

In [ ]:
scaler = preprocessing.RobustScaler()  # Definir el escalador RobustScaler
df[['METRES_CADASTRE']] = scaler.fit_transform(df[['METRES_CADASTRE']])  # Escalar la variable "METRES_CADASTRE"

TAREA: Normaliza/Escala las demÃ¡s variables continuas con MinMaxScaler o RobustScaler.<br>
Escribe el cÃ³digo a continuaciÃ³n:


S5.T3. Codificar variables categÃ³ricas: <br>
- Identifica las variables categÃ³ricas binarias, ordinales y nominales de tu base de datos.<br>
- Realiza la codificaciÃ³n de todas las variables categÃ³ricas (binarias, ordinales y nominales) que vayas a incluir en tu modelo.<br>
- Describe la estrategia de codificaciÃ³n que has empleado en cada caso.<br>


TODO 1: Identifica las variables categÃ³ricas binarias, ordinarias y nominales de tu base de datos. Completa la lista:

Variables categÃ³ricas binarias:<br>
Variables categÃ³ricas ordinales:<br>
Variables categÃ³ricas nominales:

TODO 2: Realiza la codificaciÃ³n de todas las variables categÃ³ricas binarias. Escribe el cÃ³digo a continuaciÃ³n:

EJEMPLO 1: VEHICLE ELECTRIC

In [ ]:
df["VEHICLE ELECTRIC"] = df["VEHICLE ELECTRIC"].replace({'NO': 0, 'SI': 1})  # Codificar la variable "VEHICLE ELECTRIC" con 0 y 1

TAREA: Completa la codificaciÃ³n de las demÃ¡s variables binarias. Escribe el cÃ³digo a continuaciÃ³n:

TODO 3: Realiza la codificaciÃ³n de todas las variables categÃ³ricas ordinales.

EJEMPLO 1: QualificaciÃ³ de consum d'energia primaria no renovable

In [ ]:
df["QualificaciÃ³ de consum d'energia primaria no renovable"].value_counts()  # Verificar los valores de la variable "QualificaciÃ³ de consum d'energia primaria no renovable"
ord_QualificaciÃ³_nr = {'G': 0, 'F': 1, 'E': 2, 'D': 3, 'C': 4, 'B': 5, 'A': 6}  # Definir el diccionario de codificaciÃ³n
df["QualificaciÃ³ de consum d'energia primaria no renovable"] = df["QualificaciÃ³ de consum d'energia primaria no renovable"].map(ord_QualificaciÃ³_nr) # Codificar la variable "QualificaciÃ³ de consum d'energia primaria no renovable"

Qualificacio d'emissions de CO2 (en caso de que sea vuestra variable de salida elegida)

rd_QualificaciÃ³_CO2 = {'G': 1, 'F': 2, 'E': 3, 'D': 4, 'C': 5, 'B': 6, 'A': 7}<br>
f["Qualificacio d'emissions de CO2"] = df["Qualificacio d'emissions de CO2"].map(ord_QualificaciÃ³_CO2)

TAREA: Completa la codificaciÃ³n de las demÃ¡s variables ordinales. Escribe el cÃ³digo a continuaciÃ³n:

TODO 4: Realiza la codificaciÃ³n de todas las variables categÃ³ricas nominales. Escribe el cÃ³digo a continuaciÃ³n:

EJEMPLO 1: Eina de certificaco

In [ ]:
df = pd.get_dummies(df, columns=["Eina de certificacio"])  # Codificar la variable "Eina de certificacio" con get_dummies

TAREA: Completa la codificaciÃ³n de las demÃ¡s variables nominales. Escribe el cÃ³digo a continuaciÃ³n:

TODO: Guardar base de datos codificada

In [ ]:
df = df.drop_duplicates()  # Eliminar duplicados
df.to_csv(r'E:\Laptop\BBDD_ST5.csv', index=False)  # Guardar la base de datos codificada